<a href="https://colab.research.google.com/github/wonjae124/Capston/blob/main/pipeline_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://guru.tistory.com/50

In [1]:
import pandas as pd
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns = ['sepal_length', 'sepel_width','petal_length','petal_width'])
target = iris.target
df.head()

,sepal_length,sepel_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [44]:
dir(df) # dir() 내장 함수로 attribute를 본다.

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex

TypeError: ignored

In [3]:
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC #Pipeline을 해줄 대상인 서포트 백터 머신
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(df,target,test_size = 0.2, random_state = 11)

pipe = Pipeline([('scaler', StandardScaler()), ('svc',SVC())]) #사용할 함수랑 딕셔너리 형태로 만드는듯 함
pipe.fit(X_train, Y_train) 
pipe.score(X_test, Y_test) #score method를 통해서 X_test, Y_test의 결과를 float형태로 표현


0.9333333333333333

## PipeLine parameter 넘겨주기

In [34]:
from sklearn.decomposition import PCA
estimators = [('reduce_dim',PCA()),('clf',SVC())] #왜 둘이 묶어준 것일까? estimator 처음 들어봄
pipe1 = Pipeline(estimators)
pipe1
pipe1.fit(X_train, Y_train)
pipe1.score(X_test, Y_test)

0.9666666666666667

## Params 접근하기

In [43]:
print(pipe1[0])
print(pipe1[1])
pipe1.set_params(clf__C=10) #원래는 괄호 안이 비어있는데 여길 채워줌. 근데 굳이 undersocre를 두개나 붙여주는 이유? 하나 해볼까
print(pipe1[1])


PCA()
SVC()
SVC(C=10)


## GridsearchCV로 최적 estimator 찾기

In [16]:
from sklearn.model_selection import GridSearchCV #GridSearchCV의 기능을 살펴볼 필요가 있다.
param_grid = dict(reduce_dim__n_components = [0, 1, 2], #새 변수인가?
                  clf__C = [0.1, 1, 10]
                  )

grid_search = GridSearchCV(pipe1,param_grid=param_grid)
grid_search.fit(X_train, Y_train)
grid_search.best_estimator_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 196, in fit
    accept_large_sparse=False,
  File "/usr/local/lib/python3.7/dist-packages/sklear

Pipeline(steps=[('reduce_dim', PCA(n_components=2)), ('clf', SVC(C=10))])

## 최적 estimator로 pipe.score 측정

In [18]:
pipe2 = Pipeline(steps=[('reduce_dim',PCA(n_components=2)),('clf',SVC(C=10))])
pipe2.fit(X_train,Y_train)
pipe2.score(X_test, Y_test)
# 96%에서 정답율 100%로 상향

1.0